In [1]:
import sys
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import seaborn as sns
from functools import partial
import pysam
from tqdm.notebook import trange
from tqdm.autonotebook import tqdm


import warnings

from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action='ignore', category=SettingWithCopyWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)

<ipython-input-1-7cd3a621f227>:8: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
# Import data, custom figure-making functions
sys.path.append('/mnt/d/orchards')
from figure_constants import *
from figure_functions import *
sys.path.append(installDir+'scripts')
from chartannotator import add_stat_annotation as si
multiple_annotation_method=None

loading subjects...
loading samples...
loading segments...
loading genes...
loading SNPs...
loading transmission pairs...
loading transmission segments...
loading transmission SNPs...


/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3337: DtypeWarning: Columns (58) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [3]:
real_transmissionPairs = transmissionPairs.loc[transmissionPairs.kind=='transmission']
NUMBER_OF_H3N2_PAIRS = len(real_transmissionPairs.loc[real_transmissionPairs.subtype=='H3N2'])
NUMBER_OF_H1N1_PAIRS = len(real_transmissionPairs.loc[real_transmissionPairs.subtype=='H1N1'])
NUMBER_OF_FLUB_PAIRS = len(real_transmissionPairs.loc[real_transmissionPairs.subtype=='Influenza B'])


subtype = 'H3N2'

In [4]:
import functools
def memoize(obj):
    cache = obj.cache = {}
    
    @functools.wraps(obj)
    def memoizer(*args, **kwargs):
        key = str(args) + str(kwargs)
        if key not in cache:
            cache[key] = obj(*args, **kwargs)
        return cache[key]
    return memoizer

In [5]:
@memoize
def getReadDepth(sample, segment, pos, alt):
    reffile = SNPs.loc[SNPs['sampleID']==sample, 'referenceFile'].iloc[0]
    ref = reffile.split('/')[5]
    refbase = reffile.split('/')[-1].split('_')
    if 'Hong_Kong' in reffile:
        chrom = hongkongContigs[segment]
    elif 'Michigan' in reffile:
        chrom = '_'.join(refbase[:-4])+'_'+segment
    elif refbase[-3] in ['17','18','19']:
        chrom = '_'.join(refbase[:-3])+'_'+segment
    else:
        chrom = '_'.join(refbase[:-2])+'_'+segment

    bamfile = '/'.join(reffile.split('/')[0:6])+'/'+'_'.join(reffile.split('/')[-1].split('_')[:-2])+'/map_to_consensus/'+sample+'.bam'
    pos = int(pos)
    sam = pysam.AlignmentFile(bamfile, "rb")
    try:
        pileup = sam.pileup(contig=chrom, start=pos-1, end=pos, truncate=True, stepper="nofilter")
        column = next(pileup)
    except StopIteration:
        print (chrom, pos)
        print (pileup)
        print (bamfile)
        return (0,0,0)        
    except:
        print (sam.references)
        print (chrom)
        print (reffile)
        print (ref)
        raise
    
    column.set_min_base_quality(30)
    try:
        bases = column.get_query_sequences(mark_matches=True)
        altreads = bases.count(alt.lower()) + bases.count(alt.upper())
    except:
        altreads = 0
    
    depth = column.get_num_aligned()
    if depth > 0:
        frequency = round(altreads/column.get_num_aligned(),4)
    else:
        frequency = 0
    return frequency, altreads, depth

def checkForDuplicateColumnsPostMerge(df, suffixes=('_x','_y'), verbose=False):
    '''if an index/contact or x/y column pairing are identical, unify them into one column.
       Keeps np.nan values seperate.'''
    columns = [column[:-len(suffixes[0])] for column in df.columns if column[-len(suffixes[0]):]==suffixes[0]]
    
    merged=[]
    kept = []
    for column in columns:
        columna = column+suffixes[0]
        columnb = column+suffixes[1]
                
        a=df[columna].values
        b=df[columnb].values
        
        if (df[columna].dtype.kind in 'biufc') and (df[columnb].dtype.kind in 'biufc'):
            theyAreEqual = ((a==b)|np.isclose(a,b,atol=1E-4)|np.isclose(b,a,atol=1E-4))
        else:
            theyAreEqual = ((a==b))
        if theyAreEqual.all():
            df = df.rename(columns={columna:column}).drop(columns=[columnb])
            merged.append(column)
        
        else:
            kept.append(column)
    if verbose:
        print('merged:')
        print (merged)
        print('kept:')
        print(kept)
    return df

def updateDuplicateColumnsPostMerge(df, exclude=[], suffixes=('_x','_y'), verbose=False):
    '''if an index/contact or x/y column pairing are identical except for na values, unify them into one column.
       Assumes np.nan values are artifacts, and fills in values if one column has them'''

    columns = [column[:-len(suffixes[0])] for column in df.columns if column[-len(suffixes[0]):]==suffixes[0]]

    merged=[]
    kept = []
    for column in columns:
        columna = column+suffixes[0]
        columnb = column+suffixes[1]
                
        a=df[columna].values
        b=df[columnb].values
        
        if (df[columna].dtype.kind in 'biufc') and (df[columnb].dtype.kind in 'biufc'):
            theyAreEqual = ((a==b)|pd.isna(a)|pd.isna(b)|np.isclose(a,b,atol=1E-4)|np.isclose(b,a,atol=1E-4))
        else:
            theyAreEqual = ((a==b)|pd.isna(a)|pd.isna(b))
        
        if 'AAstr' in column:
            if verbose:
                print (((a==b)|pd.isna(a)|pd.isna(b)).all())
                print (df[((a!=b)&pd.notna(a)&pd.notna(b))])
        
        if theyAreEqual.all():
            df[columna].update(df[columnb])
            df = df.rename(columns={columna:column}).drop(columns=[columnb])
            merged.append(column)
        else:
            kept.append(column)
    
    if verbose:
        print('updated:')
        print (merged)
        print('untouched:')
        print(kept)
    return df

@memoize
def getReadDepthWrapper(row):
    if pd.isna(row.SNP_frequency_index):
        try:
            result = getReadDepth(row['index'], row.segment,row.pos,row.alt_nuc)+(row.SNP_frequency_contact,row.AD_contact,row.depth_contact)
        except:
            print (row[['index','contact','segment','pos','SNP_frequency_index','SNP_frequency_contact']])
            raise
    elif pd.isna(row.SNP_frequency_contact):
        try:
            result = (row.SNP_frequency_index,row.AD_index,row.depth_index)+getReadDepth(row.contact, row.segment,row.pos,row.alt_nuc)
        except:
            print (row)  
            raise
    else:
        result = (row.SNP_frequency_index,row.AD_index,row.depth_index,row.SNP_frequency_contact,row.AD_contact,row.depth_contact)

    return result

In [6]:
def draw_rand_pairing(n):
    int1 = np.random.randint(0, n)
    int2 = np.random.randint(0, n)
    while int1 == int2:
        int2 = np.random.randint(0, n)
    return (int1, int2)

def draw_rand_pairings(n, p_candidates):
    rand_pairings = list()
    while len(rand_pairings) < n+1:
        index, contact = draw_rand_pairing(len(p_candidates))
        if abs(p_candidates.iloc[index].time_of_symptom_onset - p_candidates.iloc[contact].time_of_symptom_onset) <= pd.Timedelta(7, 'd'):
            if p_candidates.iloc[index].subclade == p_candidates.iloc[contact].subclade:
                if pd.notna(p_candidates.iloc[index].day0_sample):
                    index = p_candidates.iloc[index].day0_sample
                else:
                    index = p_candidates.iloc[index].day7_sample
                if pd.notna(p_candidates.iloc[contact].day0_sample):
                    contact = p_candidates.iloc[contact].day0_sample
                else:
                    contact = p_candidates.iloc[contact].day7_sample
                rand_pairings.append((index, contact))
    return rand_pairings

In [7]:
def add_antigenic_product(transmissionSNPs):
    HA_add_on = transmissionSNPs.loc[transmissionSNPs['product'].isin(['HA_antigenic','HA_nonantigenic'])]
    HA_add_on.loc[:, 'product'] = 'HA'
    transmissionSNPs = transmissionSNPs.append(HA_add_on)
    return transmissionSNPs

def make_all_changes_minor_to_major(transmissionSNPs):
    #Adjust SNP frequencies so that I'm always looking at the change that happens to the *minor* allele
    transmissionSNPs['minorAlleleFreq_index']= transmissionSNPs.SNP_frequency_index
    transmissionSNPs['minorAlleleFreq_contact']= transmissionSNPs.SNP_frequency_contact
    transmissionSNPs['minor_alt_nuc']= transmissionSNPs.alt_nuc
    transmissionSNPs['minor_ref_nuc']= transmissionSNPs.ref_nuc
    print (transmissionSNPs.SNP_frequency_index.max())
    tmpSNPs = transmissionSNPs.copy()

    majorityMinoritySNPs=tmpSNPs.SNP_frequency_index > 0.5
    alt_nucs = tmpSNPs.loc[majorityMinoritySNPs,'alt_nuc']
    tmpSNPs.loc[majorityMinoritySNPs,'minor_alt_nuc'] = tmpSNPs.loc[majorityMinoritySNPs,'ref_nuc']
    tmpSNPs.loc[majorityMinoritySNPs,'minor_ref_nuc'] = alt_nucs

    tmpSNPs.loc[majorityMinoritySNPs, 'minorAlleleFreq_index'] = np.abs(1-tmpSNPs.loc[majorityMinoritySNPs, 'SNP_frequency_index'].values)
    tmpSNPs.loc[majorityMinoritySNPs, 'minorAlleleFreq_contact'] = np.abs(1-tmpSNPs.loc[majorityMinoritySNPs, 'SNP_frequency_contact'].values)

    tmpSNPs['SNP_frequency_directional_change'] = tmpSNPs.SNP_frequency_contact - tmpSNPs.SNP_frequency_index
    tmpSNPs['abs_SNP_frequency_difference'] = np.abs(tmpSNPs.SNP_frequency_directional_change)
    return tmpSNPs

def calc_changes_in_SNP_frequency(df):
    df['abs_SNP_frequency_difference'] = np.abs(df.SNP_frequency_contact-df.SNP_frequency_index)
    df['SNP_frequency_directional_change'] = df.SNP_frequency_contact-df.SNP_frequency_index
    df['log_abs_SNP_frequency_difference'] = np.log10(df.abs_SNP_frequency_difference).fillna(0).replace((np.inf), 0).replace((-np.inf),0)
    return df

def apply_depth_filter(transmissionSNPs, min_depth=100):
    return transmissionSNPs.loc[~((transmissionSNPs.depth_contact < min_depth)|(transmissionSNPs.depth_index < min_depth))]
    

In [8]:
# minimum allele frequency is used as a pseudocount here when synon/nonsynon divergence in a gene pairing is 0
def calc_pairing_divergences(transmittedSNPs, pairings, subtype='H3N2', freq_cutoff=0.01):
    # First, calculate divergences only using SNPs that are above the freq cutoff
    transmittedSNPs = transmittedSNPs.loc[transmittedSNPs.abs_SNP_frequency_difference >= freq_cutoff]
    
    # Next, in order to account for gene pairings with no differences between them
    # (and thus aren't reprsented in the transmittedSNPs dataframe), 
    # I will make a separate dataframe of id columns that contain all possible pairing/gene/AAtype combinations.
    all_possible_products = pd.DataFrame(transmittedSNPs.loc[transmittedSNPs.subtype == subtype,'product'].dropna().unique())
    AAtypes = pd.DataFrame(['Nonsynonymous', 'Synonymous'])

    # in case a pairing has no differences at all, I will use a separate list of pairings
    transmittedSNPs['pairing_id'] = transmittedSNPs['index'] + '|' + transmittedSNPs.contact
    pairing_ids = pairings['index']+ '|' + pairings.contact 

    # actually create the dataframe of possible combinations
    pairing_divergence = all_possible_products.merge(AAtypes, how='cross').merge(pd.DataFrame(pairing_ids).reset_index(drop=True), how='cross')
    pairing_divergence = pairing_divergence.rename(columns={'0_x':'product','0_y':'AAtype',0:'pairing_id'})
    
    # calc the sum of absolute SNP frequency changes (aka divergence) 
    # and merge with all possible combinations of pairings/genes/AAtypes. 
    # Combinations of id variables that have no changes will be nan, and so I will set those at 0.
    between_pairing_sum_of_frequencies = transmittedSNPs.groupby(['pairing_id', 'product', 'AAtype']).sum()['abs_SNP_frequency_difference'].reset_index().rename(columns={'abs_SNP_frequency_difference':'sum_divergence'})
    pairing_divergence = pairing_divergence.merge(between_pairing_sum_of_frequencies, on=['pairing_id','product', 'AAtype'], how='left')
    pairing_divergence.sum_divergence = pairing_divergence.sum_divergence.fillna(0)
    
    # To finish off calculating the total divergence, because I will be taking the log of these data, which are counts, 
    # I have to deal with missing data (0s that are due to under-counting. Presumably 0s just represent regions that
    # rarely aquire mutations. It is difficult to observe rate of mutation below a hard cutoff of 1/len of gene.
    # One common way to deal with this is to add a pseudocount of the smallest possible observation to all observations.
    # For this experiment, the smallest possible observation is one mutation between pairs of frequency "freq_cutoff".
    # So I add that pseudocount here.
    pseudocount = freq_cutoff
    pairing_divergence.sum_divergence += pseudocount 
    
    
    # I will normalize divergences to the number of synon/nonsynon sites in the *index* case. So first, identify index:
    pairing_divergence['sampleID'] = pairing_divergence.pairing_id.str.split('|').str[0]

    # And merge with the genes dataframe to add the number of synon/nonsynon sites per gene in the index cases.
    # There should not be any missing data here; I should previously have calculated this for all samples collected.
    pairing_divergence = pairing_divergence.merge(genes[['sampleID','product','N_sites_gene','S_sites_gene']], on=['sampleID','product'], how='left').rename(columns={'sampleID':'index'})

    # Now reorganize the synon/nonsynon sites data so that each row is either synon or nonsynon only
    pairing_divergence['sites'] = pairing_divergence.N_sites_gene
    pairing_divergence.loc[pairing_divergence.AAtype == 'Synonymous', 'sites'] = pairing_divergence.loc[pairing_divergence.AAtype == 'Synonymous', 'S_sites_gene'].values
    
    # Finally, I can now normalize total divergence by the number of possible sites
    pairing_divergence['normalized_divergence'] = pairing_divergence.sum_divergence/pairing_divergence.sites
    
    # And I can calculate the log of the normalized divergece.
    pairing_divergence['log_divergence'] = np.log10(pairing_divergence.normalized_divergence)
    
    return pairing_divergence

In [9]:
from tqdm.notebook import tqdm
def get_all_pairing_divergences(pairings):
    # It is simpler to calculate the per-gene, per-AAtype normalized divergence for all possible pairings 
    # (there are ~2600 of them) than to do this for 10000 random pairs. This function does that.
    print('making pairings')
    transmissionPairs = pd.DataFrame(pairings, columns=['index','contact']).dropna()
    transmissionPairs['kind'] = 'transmission'
    
    divergence_relevent_columns =  ['sampleID',
                                    'SNP_frequency',
                                    'alt_nuc',
                                    'ref_nuc',
                                    'depth',
                                    'RD',
                                    'AD',
                                    'subtype',
                                    'AAtype',
                                    'segment',
                                    'pos',
                                    'product']
    
    print('obtaining transmitted SNPs for all pairings')
    index_SNPs = transmissionPairs.merge(SNPs[divergence_relevent_columns].rename(columns={'sampleID':'index'}),on='index', how='left')
    contact_SNPs = transmissionPairs.merge(SNPs[divergence_relevent_columns].rename(columns={'sampleID':'contact'}),on='contact', how='left')

    print ('making SNP keys')
    index_SNPs['SNPkey'] = index_SNPs['index'] + ':' + index_SNPs['contact'] + ':'+index_SNPs.segment+':'+index_SNPs.pos.astype(str)+':'+index_SNPs.alt_nuc+':'+index_SNPs['product'].fillna('OORF')
    contact_SNPs['SNPkey'] = contact_SNPs['index'] + ':' + contact_SNPs['contact'] + ':'+contact_SNPs.segment+':'+contact_SNPs.pos.astype(str)+':'+contact_SNPs.alt_nuc+':'+contact_SNPs['product'].fillna('OORF')

    print ('merging index and contact SNPs')
    transmissionSNPs = index_SNPs.merge(contact_SNPs, on='SNPkey', how='outer', suffixes=('_index','_contact'))
    transmissionSNPs = updateDuplicateColumnsPostMerge(transmissionSNPs, suffixes=('_index','_contact'))
    transmissionSNPs = transmissionSNPs.drop_duplicates()

    # Its important to have depths for both the index and contact for all single nucleotide differences between the two. 
    # I don't have this information for SNPs which arose de novo in the contact or reverted to refernece in the contact,
    # Because I don't have depth data for sites that are 100% reference.
    # So I will applying a function to all transmission SNPs that:
       # a) determines whether the index or contact frequency/AD/depth info contains nans
       # b) calls getReadDepth on the appropriate information to fill in the nans
       # c) returns the original data with getReadDepth's results filling in the nans
    columnsToUpdate = ['SNP_frequency_index','AD_index','depth_index','SNP_frequency_contact','AD_contact','depth_contact']

    print ('getting depths')
    tqdm.pandas()
    if os.path.exists('tSNPs_w_depth.csv'):
        tmp=pd.read_csv('tSNPs_w_depth.csv').drop('Unnamed: 0', axis=1)
    else:
        tmp = pd.DataFrame(transmissionSNPs.progress_apply(getReadDepthWrapper,axis=1).to_list())
        tmp.to_csv('tSNPs_w_depth.csv')
    #It makes me nervous that I'm applying a function to all my values which in theory could change all my SNP values.
    #So I'm going to do this carefully. I will apply the function and create a separate data frame, preserving my original data.
    #I then assert that the data that I am about to change is either a) identical to the new data, or b) nan
    print('checking results')
    a = transmissionSNPs[columnsToUpdate].to_numpy()
    b = tmp.to_numpy()

#     assert ((a==b) | np.isnan(a)).all()

    #Once that's confirmed, I replace my original data with my updated data
    transmissionSNPs[columnsToUpdate] = b

    # To save time, I do not get reference SNP depth, just total depth and alt depth. 
    # Both are calculated w/ quality minimums, so ref_depth is just total depth - alt depth
    # I'm only changing values that are nan, otherwise I will use the info previously gathered
    transmissionSNPs.loc[transmissionSNPs.RD_index.isna(), 'RD_index'] = transmissionSNPs.loc[transmissionSNPs.RD_index.isna(), 'depth_index']-transmissionSNPs.loc[transmissionSNPs.RD_index.isna(), 'AD_index']
    transmissionSNPs.loc[transmissionSNPs.RD_contact.isna(), 'RD_contact'] = transmissionSNPs.loc[transmissionSNPs.RD_contact.isna(), 'depth_contact']-transmissionSNPs.loc[transmissionSNPs.RD_contact.isna(), 'AD_contact']

    assert(len(transmissionSNPs.loc[transmissionSNPs.RD_index.isna()])==0)
    assert(len(transmissionSNPs.loc[transmissionSNPs.RD_contact.isna()])==0)
    assert(len(transmissionSNPs.loc[transmissionSNPs.AD_index.isna()])==0)
    assert(len(transmissionSNPs.loc[transmissionSNPs.AD_contact.isna()])==0)

    # And now that my nans are filled in, I calculate the differences in snp frequency
    print('calculating divergences')
    
    # First filter out any transmission SNPs where the index or contact was not sequenced at 
    # sufficient depth to be confident of the within-host frequency of that site
    transmissionSNPs = apply_depth_filter(transmissionSNPs, min_depth=100)
    transmissionSNPs = calc_changes_in_SNP_frequency(transmissionSNPs)

    if len(transmissionSNPs['product'] == 'HA') == 0:
        print ('acutally using this clause')
        transmissionSNPs = add_antigenic_product(transmissionSNPs)
        
    divergences = calc_pairing_divergences(transmissionSNPs, pairings=pairings)
    divergences = divergences[['pairing_id','product','AAtype','normalized_divergence']]
    return divergences

In [10]:
# First, get the normalized divergences of all combinations of samples that could plausibly be the result of 
# a transmission (ie., sx onset in contact occured within 10 days after sx onset of index)

a = allvsall.loc[allvsall.subtype_index==subtype]

# All vs all is a df of all potential pairings w/ distances pre-calculated.
# It should already be limited to plausible transmissions.
assert len(a.loc[np.abs(pd.to_datetime(a.time_of_symptom_onset_index)-pd.to_datetime(a.time_of_symptom_onset_index))<=(pd.to_timedelta('10D'))]) == len(a)

all_plausible_pairing_divergences = get_all_pairing_divergences(a[['index','contact']])

# Now that I have a df with the divergences of each AA type of each product in every plausible sample combination,
# I need to calculate the stat I'm actually interested in: the log of the ratio of Nonsynon to Synon divergences 
# for each gene product in each plausible sample combo.
# First, take the log of the normalized divergence
all_plausible_pairing_divergences['log_normalized_divergence'] = np.log(all_plausible_pairing_divergences.normalized_divergence)

# Then, do this sort of odd code that is very fast. Sort by pairing_id, product, and AA type.
all_plausible_pairing_divergences = all_plausible_pairing_divergences.sort_values(['pairing_id','product', 'AAtype']).reset_index(drop=True)

# Because I sort by AA type last, every even row is Nonsynonymous, and every odd row is Synonymous.
# So make a data frame that is just id values (ie, pairings and products):
pairing_divergences = all_plausible_pairing_divergences.groupby(['pairing_id','product']).first().reset_index()[['pairing_id','product']]
# and our log divergence ratio will be log evens = log odds. Having previously sorted by pairing_id and product, 
# the resulting numbers should be in the right order.
pairing_divergences['log_divergence_ratio'] = all_plausible_pairing_divergences.loc[all_plausible_pairing_divergences.index%2==0, 'log_normalized_divergence'].values - all_plausible_pairing_divergences.loc[all_plausible_pairing_divergences.index%2==1, 'log_normalized_divergence'].values


making pairings
obtaining transmitted SNPs for all pairings
making SNP keys
merging index and contact SNPs
getting depths
checking results
calculating divergences


In [11]:
import scipy.stats as st
###Create random pairs with distances that are in *same distribution* as household pairs

# add information about pairing genetic distances back to the all_plausible_pairing_divergences df
distances = allvsall[['index','contact','distance']]
distances['pairing_id'] = allvsall['index']+'|'+allvsall.contact

all_plausible_pairing_divergences = all_plausible_pairing_divergences.merge(distances[['pairing_id','index','contact','distance']], on='pairing_id',how='left')

# bin those pairings by distance
subtype_ava = all_plausible_pairing_divergences.groupby(['pairing_id','distance']).first().reset_index()[['pairing_id','distance']]
subtype_ava['quantiles'] = pd.cut(subtype_ava.distance,bins=np.linspace(0, 500, 50001), labels=np.linspace(0.01, 500, 50000))

subtype_transPairs = transmissionPairs.loc[(transmissionPairs.subtype==subtype) & (transmissionPairs.kind=='transmission')].reset_index(drop=True)

bootstrap_size = 10000
bootstrap_stat = 'distance'

# then fit a log-norm distribution to our actual transmissionPairs to match distances to
mu, sigma = np.log(subtype_transPairs.distance).mean(), np.log(subtype_transPairs.distance).std()

In [12]:
def draw_n_random_pairings_from_lognormal_distribution(potential_pairings, n, mu, sigma):
    random_pairings = list()
    print('taking distance distribution')
    samples = np.round(np.random.lognormal(mean=mu, sigma=sigma, size=n), 2)
    print('finding samples w/ distances nearest to drawn distances')
    potentials = potential_pairings.quantiles.cat.categories[potential_pairings.quantiles.cat.codes].values
    
    adjusted_samples = potentials[np.abs(samples[:,np.newaxis]-potentials[np.newaxis,:]).argmin(axis=1)]
    print('darwing bootstrapped sample pairs')
    random_pairings = [potential_pairings.loc[potential_pairings.quantiles==x].sample(1)['pairing_id'].values[0] for x in adjusted_samples]

    return random_pairings

In [13]:
# its faster to draw boostrapSize*numOfBootstraps random pairs from the lognormal distribution 
# and later reshape into a [boostrapSize, numOfBootstraps] shaped dataframe/array
drawings = draw_n_random_pairings_from_lognormal_distribution(potential_pairings = subtype_ava,
                                                              n = NUMBER_OF_H3N2_PAIRS*bootstrap_size,
                                                              mu=mu,
                                                              sigma=sigma) 

taking distance distribution
finding nearest samples
getting pairings


In [14]:
random_drawings_df = pd.DataFrame(drawings, columns=['pairing_id'])

random_drawings_df['bootstrap_id'] = random_drawings_df.index % 10000

# Now that I have 10000 bootstraps of n pairs, 
# I can merge with pairing_divergences to get the log divergence ratios of each product for all my randomly drawn pairs
random_drawings_df = random_drawings_df.merge(pairing_divergences, on='pairing_id', how='left')

# Calc the average log divergence ratio for each product per bootstrap
random_drawings_df = random_drawings_df.groupby(['bootstrap_id', 'product']).mean().reset_index()

# And save the product. The rest of the work will be done in the notebook that actually makes the figure.
random_drawings_df.to_csv('/mnt/d/orchards/10000_random_H3N2_log_ratios.tsv', sep='\t')